# Example: Optimal adversaries for dense MNIST model


## Building and training the neural network

In [2]:
#Import requisite packages
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

Show how to load the dataset for training

In [3]:
#Set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#Build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Define model

In [4]:
hidden_size = 50

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1  = nn.Conv2d(1, 8, (4,4), (2,2), 0)
        self.conv2  = nn.Conv2d(8, 8, (4,4), (2,2), 0)
        self.hidden1 = nn.Linear(5 * 5 * 8, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        self.x1 = self.conv1(x)
        self.x2 = self.relu(self.x1)
        self.x3 = self.conv2(self.x2)
        self.x4 = self.relu(self.x3)
        self.x5 = self.hidden1(self.x4.view((-1,5*5*8)))
        self.x6 = self.relu(self.x5)
        self.x7 = self.output(self.x6)
        x = self.softmax(self.x7)      
        return x

Define train and test functions

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0; correct = 0
    criterion = nn.NLLLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Train model on dataset

In [8]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.294026
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.259291
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.041207
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.110864
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.190586

Test set: Average loss: 0.0924, Accuracy: 9700/10000 (97%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.014803
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.078992
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.054852
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.166482
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.031017

Test set: Average loss: 0.0719, Accuracy: 9757/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.040141
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.024642
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.009056
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.120740
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.013258

Test set: Average loss: 0.0569, Accuracy: 9809/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.053137
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Building the MIP formulation

Need to export to ONNX, the PyTorch ONNX exporter needs to write to a file so we generate a temporary file.

In [9]:
import torch.onnx
import tempfile
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

We also define bounds on variables

In [10]:
problem_index = 0
image = dataset2[problem_index][0].detach().numpy()
label = dataset2[problem_index][1]

In [30]:
epsilon_infty = 5e-2
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)

input_bounds = {}
for i in range(28):
    for j in range(28):
        input_bounds[(0,i,j)] = (float(lb[0][i,j]), float(ub[0][i,j])) 

PyTorch needs to trace the model execution to export it, so we defined a dummy input tensor.

In [19]:
x = dataset2[problem_index][0].view(-1,1,28,28)

Now we can write the ONNX model and load it back.

In [17]:
class NoSoftmaxNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1  = nn.Conv2d(1, 8, (4,4), (2,2), 0)
        self.conv2  = nn.Conv2d(8, 8, (4,4), (2,2), 0)
        self.hidden1 = nn.Linear(5 * 5 * 8, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        self.x1 = self.conv1(x)
        self.x2 = self.relu(self.x1)
        self.x3 = self.conv2(self.x2)
        self.x4 = self.relu(self.x3)
        self.x5 = self.hidden1(self.x4.view((-1,5*5*8)))
        self.x6 = self.relu(self.x5)
        self.x7 = self.output(self.x6)
        x = self.softmax(self.x7)      
        return x
    
model2 = NoSoftmaxNet()
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

In [31]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    torch.onnx.export(
        model2,
        x,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load back
    network_definition = load_onnx_neural_network_with_bounds(f.name)

Create Pyomo model

In [21]:
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import NeuralNetworkFormulation

In [22]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[1, 28, 28], output_size=[1, 28, 28])	linear
1	ConvLayer(input_size=[1, 28, 28], output_size=[8, 13, 13], strides=[2, 2], kernel_shape=(4, 4))	relu
2	ConvLayer(input_size=[8, 13, 13], output_size=[8, 5, 5], strides=[2, 2], kernel_shape=(4, 4))	relu
3	DenseLayer(input_size=[1, 200], output_size=[1, 50])	relu
4	DenseLayer(input_size=[1, 50], output_size=[1, 10])	logsoftmax


OMLT doesn't include a formulation for sigmoid, so define it here

In [32]:
formulation = NeuralNetworkFormulation(network_definition)
    #activation_constraints={'relu': relu_activation}
#)

m = pyo.ConcreteModel()

m.nn = OmltBlock()
m.nn.build_formulation(formulation) 

AttributeError: '_BlockData' object has no attribute 'constraints'

In [29]:
m.obj = pyo.Objective(expr=(-(m.nn.outputs[label+1]-m.nn.outputs[label])))

In [31]:
pyo.SolverFactory('gurobi').solve(m, tee=True)

Academic license - for non-commercial use only - expires 2023-01-12
Using license file /Users/calvintsay/gurobi.lic
Read LP format model from file /var/folders/pc/7mzx4b956_lb2l8_ryngwydc0000gn/T/tmp7ns5p94m.pyomo.lp
Reading time = 0.04 seconds
x2693: 2109 rows, 2693 columns, 46307 nonzeros
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2109 rows, 2693 columns and 46307 nonzeros
Model fingerprint: 0xb20823a5
Variable types: 2593 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-03, 8e+01]
  RHS range        [3e-04, 1e+01]
Presolve removed 1831 rows and 1696 columns
Presolve time: 0.09s
Presolved: 278 rows, 997 columns, 30454 nonzeros
Variable types: 932 continuous, 65 integer (65 binary)

Root relaxation: objective -8.661667e+00, 140 iterations, 0.01 seconds

    Nodes    |    Curr

{'Problem': [{'Name': 'x2693', 'Lower bound': 3.9967748711641224, 'Upper bound': 3.996774871164125, 'Number of objectives': 1, 'Number of constraints': 2109, 'Number of variables': 2693, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of continuous variables': 2593, 'Number of nonzeros': 46307, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '1.1556038856506348', 'Error rc': 0, 'Time': 1.2740771770477295}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}